# Workload Summary 

Find out the top 10 videos with fastest growth
of dislikes number between its first and second trending appearances. Here we measure
the growth of dislikes number by the gap of dislikes increase and likes increase
between the first two trending appearances in the same country.

For instance, the dislikes growth of video QwZT7T-TXT0 in US is computed as follows:

```
(1065772 - 629120) - (1082422 - 835378) = 189608
```

Where the first component is the increase of dislikes and the second component is
the increase of likes between the first and second trending appearances .

The result of this workload should show a few details of the top 10 videos, including the
video id, category, dislike growth value and country code. Below is a few sample results:

```
"BEePFpC9qG8", 366556, "Film & Animation", "DE"
"RmZ3DPJQo2k", 334594, "Music", "KR"
"1Aoc-cd9eYs", 192222, "Entertainment", "GB"
"QwZT7T-TXT0", 189608, "Entertainment", "US"
"QwZT7T-TXT0", 189605, "Entertainment", "GB"
```

If a video has changed its category name over time, you can use the category of the
first appearance. It is possible to include the same video multiple times in top 10 list if it
has large dislikes growth in multiple countries. Video QwZT7T-TXT0 is such an example.

## Required Imports 
import necessary packages and expose pyspark to Jupyter Notebook.

In [1]:
# all required import libraries 
import findspark
# from ml_utils import *
findspark.init()
from pyspark import SparkContext

## Create spark context and set paths 
create the spark context and set the input/output paths 

In [3]:
sc = SparkContext(appName="Top 10 Fastest Video Dislikes Trend")

#You can change the input path pointing to your own HDFS
#If spark is able to read hadoop configuration, you can use relative path
input_path = '../data/' # local file system relative path 

#Relative path is used to specify the output directory
#The relative path is always relative to your home directory in HDFS: /user/<yourUserName>
output_path = 'top10VideosOut'

## load the datasets 

In [4]:
videoData = sc.textFile(input_path + "AllVideos_short.csv")

## Run the filtering and processing 

In [5]:
print(type(videoData))

#movieRatings = ratings.map(extractRating)
#movieGenre = movieData.flatMap(pairMovieToGenre) # we use flatMap as there are multiple genre per movie

#genreRatings = movieGenre.join(movieRatings).values()
#genreRatingsAverage = genreRatings.aggregateByKey((0.0,0), mergeRating, mergeCombiners, 1).map(mapAverageRating)



<class 'pyspark.rdd.RDD'>


In [ ]:
# save the output 
#genreRatingsAverage.saveAsTextFile(output_path)

## Final step - safely stop the spark context

In [6]:
# stop the spark context 
sc.stop()